In [2]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [3]:
import pandas as pd
import numpy as np
import math
import random

# Examine sensors metadata to create adjacency matrix

In [4]:
sensors = pd.read_csv('/content/d07_text_meta_2022_07_15.txt', sep="\t")

In [5]:
def measure(lat1, lon1, lat2, lon2) :
    R = 6378.137 #Radius of earth in KM
    dLat = lat2 * math.pi / 180 - lat1 * math.pi / 180
    dLon = lon2 * math.pi / 180 - lon1 * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180) * math.cos(lat2 * math.pi / 180) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c;
    return d * 1000  #meters

In [6]:
def sample_dataset(df, n_samples, max_range) :
  # need to sample data in order to keep in a limited range, prob = 1 / df.shape[0]
  #sampled_df = df.sample(n = n_samples)
  sampled_df = np.zeros([n_samples, 3])
  distances = np.zeros([n_samples, n_samples])
  r = random.randint(0, df.shape[0])
  lat_r = df.iloc[r]['Latitude']
  lon_r = df.iloc[r]['Longitude']
  sampled_df[0] = [df.iloc[r]['ID'], df.iloc[r]['Latitude'], df.iloc[r]['Longitude']]
  count = 1
  sensors_list = [r]
  while count != n_samples:
    row = random.randint(0, df.shape[0])
    if row not in sensors_list:
      sensors_list.append(row)
      lat_i = df.iloc[row]['Latitude']
      lon_i = df.iloc[row]['Longitude']
      if 0 < measure(lat_r, lon_r, lat_i, lon_i) < max_range:
        sampled_df[count] = [df.iloc[row]['ID'], df.iloc[row]['Latitude'], df.iloc[row]['Longitude']]
        count +=1
        

  for i in range(n_samples):
    for j in range(n_samples):
      if i != j :
        # convert from lat/long to distance in meters
        lat_i = sampled_df[i][1]
        lat_j = sampled_df[j][1]
        lon_i = sampled_df[i][2]
        lon_j = sampled_df[j][2]
        distances[i, j] = measure(lat_i, lon_i, lat_j, lon_j)

  
  return pd.DataFrame(sampled_df, columns=['ID', 'Latitude', 'Longitude']), pd.DataFrame(distances)


In [7]:
sampled_df, distances = sample_dataset(sensors, 228, 20_000)

In [10]:
display(max(distances.max()))

39131.06797258615

# Build the speed measurement dataset (Alberto)